# Infant resting state fMRI preprocessing
This notebook contains preprocessing tailored to infant resting state fMRI collected in 5-8 month olds. 

The processing steps for the fMRI broadly include:
* Slice-time correction
* Rigid realignment
* Co-registration to the sMRI (T2-weighted structural MRI)
* Artifact detection:
    - Motion
    - Global intensity outliers
* De-noising to remove:
    - Component noise associated with white matter and CSF
    - component noise associated with motion
    - Censoring/scrubbing of individual volumes detected as artifacts in the previous step
    - Frame-wise displacement
* Bandpass filtering
* Spatial smoothing
* Registration to infant sample template

In [1]:
#import packages
from os import listdir, makedirs
from os.path import isdir
from nipype.interfaces.io import DataSink, SelectFiles # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim

from nipype.algorithms.rapidart import ArtifactDetect
from nipype.interfaces.fsl.preprocess import SliceTimer, MCFLIRT, FLIRT, FAST, SUSAN
from nipype.interfaces.fsl.utils import Reorient2Std, MotionOutliers
from nipype.interfaces.fsl.model import GLM
from nipype.interfaces.fsl.maths import ApplyMask, TemporalFilter
from nipype.interfaces.freesurfer import Resample, Binarize, MRIConvert
from nipype.algorithms.confounds import CompCor
from nipype.interfaces.afni.preprocess import Bandpass
from nipype.interfaces.afni.utils import AFNItoNIFTI
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.ants import Registration, ApplyTransforms
from pandas import DataFrame, Series

#set output file type for FSL to NIFTI
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
studyhome = '/Users/catcamacho/Box/SNAP/BABIES'
#studyhome = '/Volumes/iang/active/BABIES/BABIES_rest'
raw_data = studyhome + '/raw'
output_dir = studyhome + '/proc/rest_preproc'
workflow_dir = studyhome + '/workflows'
#subjects_list = open(studyhome + '/misc/subjects.txt').read().splitlines()
subjects_list = ['021']

template_brain = studyhome + '/templates/T2w_BABIES_template_2mm.nii.gz'
template_wm = studyhome + '/templates/BABIES_wm_mask_2mm.nii.gz'
template_mask = studyhome + '/templates/T2w_BABIES_template_2mm_mask.nii.gz'

proc_cores = 2 # number of cores of processing for the workflows

vols_to_trim = 4
interleave = False
TR = 2.5 # in seconds
slice_dir = 3 # 1=x, 2=y, 3=z
resampled_voxel_size = (2,2,2)
fwhm = 4 #fwhm for smoothing with SUSAN

#changed to match Pendl et al 2017 (HBM)
highpass_freq = 0.08 #in Hz
lowpass_freq = 0.1 #in Hz

mask_erosion = 1
mask_dilation = 1

In [2]:
## File handling Nodes

# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name="infosource")
infosource.iterables = ('subject_id', subjects_list)


# Data grabber- select fMRI and sMRI
templates = {'struct': raw_data + '/{subject_id}-BABIES/skullstripped_anat.nii.gz',
            'func': raw_data + '/{subject_id}-BABIES/rest_raw.nii.gz'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Datasink- where our select outputs will go
substitutions = [('_subject_id_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

In [3]:
## Nodes for preprocessing

# Reorient to standard space using FSL
reorientfunc = Node(Reorient2Std(), name='reorientfunc')
reorientstruct = Node(Reorient2Std(), name='reorientstruct')

# convert files to nifti
reslice_struct = Node(MRIConvert(out_type='niigz',
                                 conform_size=2,
                                 crop_size=(128, 128, 128),
                                ),
                   name='reslice_struct')
# Segment structural scan
segment = Node(FAST(no_bias=True, 
                    segments=True, 
                    number_classes=3), 
               name='segment')

# Trim first 4 volumes using nipype 
trimvols = Node(Trim(begin_index=vols_to_trim), name='trimvols')

#Slice timing correction based on interleaved acquisition using FSL
slicetime_correct = Node(SliceTimer(interleaved=interleave, 
                                    slice_direction=slice_dir,
                                   time_repetition=TR),
                            name='slicetime_correct')

# Motion correction- MEL
motion_correct = Node(MCFLIRT(save_plots=True, 
                              mean_vol=True), 
                      name='motion_correct')

# Get frame-wise displacement for each run: in_file; out_file, out_metric_plot, out_metric_values
get_FD = Node(MotionOutliers(metric = 'fd',
                             out_metric_values = 'FD.txt',
                             out_metric_plot = 'motionplot.png',
                             no_motion_correction=False),
                 name='get_FD',)

# register BOLD to anat
coregT2 = Node(Registration(args='--float',
                            collapse_output_transforms=True,
                            initial_moving_transform_com=True,
                            num_threads=1,
                            output_inverse_warped_image=True,
                            output_warped_image=True,
                            sigma_units=['vox']*3,
                            transforms=['Rigid', 'Affine', 'SyN'],
                            terminal_output='file',
                            winsorize_lower_quantile=0.005,
                            winsorize_upper_quantile=0.995,
                            convergence_threshold=[1e-06],
                            convergence_window_size=[10],
                            metric=['Mattes', 'Mattes', 'CC'],
                            metric_weight=[1.0]*3,
                            number_of_iterations=[[100, 75, 50],
                                                  [100, 75, 50],
                                                  [70, 50, 20]],
                            radius_or_number_of_bins=[32, 32, 4],
                            sampling_percentage=[0.25, 0.25, 1],
                            sampling_strategy=['Regular',
                                               'Regular',
                                               'None'],
                            shrink_factors=[[4, 2, 1]]*3,
                            smoothing_sigmas=[[2, 1, 0]]*3,
                            transform_parameters=[(0.1,),
                                                  (0.1,),
                                                  (0.1, 3.0, 0.0)],
                            use_histogram_matching=False,
                            write_composite_transform=True,
                            dimension=3),
               name='coregT2')

# apply transform to func
applyT2xform = Node(ApplyTransforms(reference_image=template_brain, 
                                    dimension=4), 
                    name = 'applyT2xform')

# register anat to template
reg_temp = Node(Registration(args='--float',
                             collapse_output_transforms=True,
                             initial_moving_transform_com=True,
                             num_threads=1,
                             output_inverse_warped_image=True,
                             output_warped_image=True,
                             sigma_units=['vox']*3,
                             transforms=['Rigid', 'Affine', 'SyN'],
                             terminal_output='file',
                             winsorize_lower_quantile=0.005,
                             winsorize_upper_quantile=0.995,
                             convergence_threshold=[1e-06],
                             convergence_window_size=[10],
                             metric=['Mattes', 'Mattes', 'CC'],
                             metric_weight=[1.0]*3,
                             number_of_iterations=[[100, 75, 50],
                                                   [100, 75, 50],
                                                   [70, 50, 20]],
                             radius_or_number_of_bins=[32, 32, 4],
                             sampling_percentage=[0.25, 0.25, 1],
                             sampling_strategy=['Regular',
                                                'Regular',
                                                'None'],
                             shrink_factors=[[4, 2, 1]]*3,
                             smoothing_sigmas=[[2, 1, 0]]*3,
                             transform_parameters=[(0.1,),
                                                   (0.1,),
                                                   (0.1, 3.0, 0.0)],
                             use_histogram_matching=False,
                             write_composite_transform=True, 
                             fixed_image=template_brain),
                name='reg_temp')

# apply transform to func
applyxform = Node(ApplyTransforms(reference_image=template_brain, 
                                  dimension=4), 
                  name = 'applyxform')

# unzip the nifti for ART
gunzip = Node(Gunzip(), name='gunzip')

# Artifact detection for scrubbing/motion assessment
art = Node(ArtifactDetect(mask_type='file',
                          parameter_source='FSL',
                          norm_threshold=0.25, #mutually exclusive with rotation and translation thresh
                          zintensity_threshold=2,
                          use_differences=[True, False], 
                          mask_file=template_mask),
           name='art')

In [4]:
# Data QC nodes
def create_coreg_plot(epi,anat):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = os.path.abspath(coreg_filename)
    
    return(coreg_file)

def check_mask_coverage(epi,brainmask):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    maskcheck_filename='maskcheck.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'brainmask coverage')
    display.add_contours(brainmask,levels=[.5], colors='r')
    display.savefig(maskcheck_filename)
    display.close()
    maskcheck_file = os.path.abspath(maskcheck_filename)

    return(maskcheck_file)

make_coreg_img = Node(name='make_coreg_img',
                      interface=Function(input_names=['epi','anat'],
                                         output_names=['coreg_file'],
                                         function=create_coreg_plot))

make_checkmask_img = Node(name='make_checkmask_img',
                      interface=Function(input_names=['epi','brainmask'],
                                         output_names=['maskcheck_file'],
                                         function=check_mask_coverage))
make_checkmask_img.inputs.brainmask = template_mask

In [5]:
## Preprocessing Workflow

# workflowname.connect([(node1,node2,[('node1output','node2input')]),
#                    (node2,node3,[('node2output','node3input')])
#                    ])

preprocwf = Workflow(name='preprocwf')
preprocwf.connect([(infosource,selectfiles,[('subject_id','subject_id')]), 
                   (selectfiles,reorientstruct,[('struct','in_file')]),
                   (selectfiles,reorientfunc,[('func','in_file')]),
                   (reorientstruct,reslice_struct,[('out_file','in_file')]),
                   (reslice_struct,coregT2,[('out_file','fixed_image')]),
                   (reorientfunc,trimvols,[('out_file','in_file')]),
                   (trimvols,slicetime_correct,[('out_file','in_file')]),
                   (slicetime_correct,motion_correct,[('slice_time_corrected_file','in_file')]),
                   (slicetime_correct,get_FD,[('slice_time_corrected_file','in_file')]),
                   (motion_correct,coregT2,[('out_file','moving_image')]),
                   (motion_correct,art,[('par_file','realignment_parameters')]),
                   (coregT2,make_coreg_img,[('warped_image','epi')]),
                   (coregT2,applyT2xform,[('composite_transform','transforms')]),
                   (coregT2, applyT2xform,[('warped_image','input_image')]),
                   (applyT2xform, applyxform,[('output_image','input_image')]),
                   (reslice_struct,make_coreg_img,[('out_file','anat')]),
                   (applyxform,make_checkmask_img,[('output_image','epi')]),
                   (reslice_struct,reg_temp,[('out_file','moving_image')]),
                   (reg_temp,applyxform,[('composite_transform','transforms')]),
                   (applyxform,gunzip,[('output_image','in_file')]),
                   (gunzip,art,[('out_file','realigned_files')]),
                   (reg_temp,segment,[('warped_image','in_files')]),
                   
                   (applyxform, datasink, [('output_image','preproc_func')]),
                   (reg_temp, datasink,[('warped_image','preproc_anat')]),
                   (get_FD, datasink, [('out_metric_values','FD_out_metric_values')]),
                   (motion_correct,datasink,[('par_file','motion_params')]),
                   (segment,datasink,[('tissue_class_files','tissue_class_files')]),
                   (art,datasink, [('plot_files','art_plot_files')]),
                   (art,datasink, [('outlier_files','vols_to_censor')]),
                   (make_checkmask_img,datasink,[('maskcheck_file','maskcheck_image')]),
                   (make_coreg_img,datasink,[('coreg_file','coreg_image')])                   
                  ])
preprocwf.base_dir = workflow_dir
preprocwf.write_graph(graph2use='flat')
preprocwf.run('MultiProc', plugin_args={'n_procs': proc_cores})

180618-08:26:23,631 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocwf/graph.png (graph2use=flat, simple_form=True).
180618-08:26:23,872 workflow INFO:
	 Workflow preprocwf settings: ['check', 'execution', 'logging', 'monitoring']
180618-08:26:23,903 workflow INFO:
	 Running in parallel.
180618-08:26:23,909 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2.
180618-08:26:23,998 workflow INFO:
	 [Node] Setting-up "preprocwf.selectfiles" in "/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocwf/_subject_id_021/selectfiles".
180618-08:26:24,19 workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
180618-08:26:24,37 workflow INFO:
	 [Node] Finished "preprocwf.selectfiles".
180618-08:26:25,909 workflow INFO:
	 [Job 0] Completed (preprocwf.selectfiles).
180618-08:26:25,912 workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


180618-09:13:43,938 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocwf/_subject_id_021/make_coreg_img/coregistration.png;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocwf/_subject_id_021/make_coreg_img/_0xd0a9025a3857918c1bbf3a3bd3a2a1c8_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocwf/_subject_id_021/make_coreg_img/_inputs.pklz;/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocwf/_subject_id_021/make_coreg_img/_node.pklz
180618-09:13:43,940 workflow DEBUG:
	 Needed dirs: /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocwf/_subject_id_021/make_coreg_img/_report
180618-09:13:43,942 workflow DEBUG:
	 Removing files: 
180618-09:13:43,948 workflow DEBUG:
	 saved results in /Users/catcamacho/Box/SNAP/BABIES/workflows/preprocwf/_subject_id_021/make_coreg_img/result_make_coreg_img.pklz
180618-09:13:43,950 workflow DEBUG:
	 [Node] Writing post-exec report to "/Users/catcamacho/Box/SNAP/BABIES/workflows/preprocwf/_subject_id_021/mak

RuntimeError: Workflow did not execute cleanly. Check log for details

In [ ]:
# Resting state preprocessing
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subjects_list)


# Data grabber- select fMRI and sMRI
templates = {'func': output_dir + '/masked_func/{subject_id}/rest_raw_reoriented_trim_st_mcf_flirt_masked.nii.gz',
             'csf': output_dir + '/tissue_class_files/{subject_id}/skullstripped_anat_reoriented_resample_seg_0.nii.gz', 
             'vols_to_censor':output_dir + '/vols_to_censor/{subject_id}/art.rest_raw_reoriented_trim_st_mcf_flirt_masked_outliers.txt.gz', 
             'motion_params':output_dir + '/FD_out_metric_values/{subject_id}/FD.txt',
             'wm':template_wm}
selectfiles = Node(SelectFiles(templates), name='selectfiles')


In [ ]:
## Pull motion info for all subjects

motion_df = DataFrame(columns=['meanFD','maxFD','NumCensoredVols'])

if isdir(output_dir + '/motion_summary') ==False:
    makedirs(output_dir + '/motion_summary')
    
motion_df_file = output_dir + '/motion_summary/motionSummary.csv'
motion_df.to_csv(motion_df_file)

def summarize_motion(motion_df_file, motion_file, vols_to_censor):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import dirname, basename
    from numpy import asarray, mean
    from pandas import DataFrame, Series, read_csv
    
    motion_df = read_csv(motion_df_file, index_col=0)
    
    motion = asarray(open(motion_file).read().splitlines()).astype(float)
    censvols = open(vols_to_censor).read().splitlines()

    fp = dirname(motion_file)
    subject = basename(fp)

    motion_df.loc[subject] = [mean(motion),max(motion),len(censvols)]
    motion_df.to_csv(motion_df_file)

    return()

# Make a list of tissues for component noise removal
def combine_masks(mask1,mask2):
    from nipype.interfaces.fsl.utils import Merge
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    vols = []
    vols.append(mask1)
    vols.append(mask2)
    
    return(vols)
    
# Remove all noise (GLM with noise params)
def create_noise_matrix(vols_to_censor,motion_params,comp_noise):
    from numpy import genfromtxt, zeros,concatenate, savetxt
    from os import path

    motion = genfromtxt(motion_params, delimiter=' ', dtype=None, skip_header=0)
    comp_noise = genfromtxt(comp_noise, delimiter='\t', dtype=None, skip_header=1)
    censor_vol_list = genfromtxt(vols_to_censor, delimiter='\t', dtype=None, skip_header=0)

    c = len(censor_vol_list)
    d = len(comp_noise)
    if c > 0:
        scrubbing = zeros((d,c),dtype=int)
        for t in range(0,c):
            scrubbing[censor_vol_list[t]][t] = 1    
        noise_matrix = concatenate([motion[:,None],comp_noise,scrubbing],axis=1)
    else:
        noise_matrix = concatenate((motion[:,None],comp_noise),axis=1)

    noise_file = 'noise_matrix.txt'
    savetxt(noise_file, noise_matrix, delimiter='\t')
    noise_filepath = path.abspath(noise_file)
    
    return(noise_filepath)

def convertafni(in_file):
    from nipype.interfaces.afni.utils import AFNItoNIFTI
    from os import path
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    cvt = AFNItoNIFTI()
    cvt.inputs.in_file = in_file
    cvt.inputs.out_file = 'func_filtered.nii.gz'
    cvt.run()
    
    out_file = path.abspath('func_filtered.nii.gz')
    return(out_file)

# Brightness threshold should be 0.75 * the contrast between the median brain intensity and the background
def brightthresh(func):
    import nibabel as nib
    from numpy import median, where
    
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    func_nifti1 = nib.load(func)
    func_data = func_nifti1.get_data()
    func_data = func_data.astype(float)
    
    brain_values = where(func_data > 0)
    median_thresh = median(brain_values)
    bright_thresh = 0.75 * median_thresh
    
    return(bright_thresh)

In [ ]:
# Denoising
merge_confs = Node(Function(input_names=['mask1','mask2'],
                            output_names=['vols'], 
                            function=combine_masks), 
                   name='merge_confs')

compcor = Node(CompCor(merge_method='none'), 
               name='compcor')

noise_mat = Node(Function(input_names=['vols_to_censor','motion_params','comp_noise'],
                          output_names=['noise_filepath'], 
                          function=create_noise_matrix), 
                 name='noise_mat')

denoise = Node(GLM(out_res_name='denoised_residuals.nii', 
                   out_data_name='denoised_func.nii'), 
               name='denoise')

# band pass filtering- all rates are in Hz (1/TR or samples/second)
bandpass = Node(Bandpass(highpass=highpass_freq,
                         lowpass=lowpass_freq), 
                name='bandpass')

afni_convert = Node(Function(input_names=['in_file'],
                             output_names=['out_file'],
                             function=convertafni), 
                    name='afni_convert')

# Spatial smoothing 
brightthresh_filt = Node(Function(input_names=['func'], 
                                  output_names=['bright_thresh'], 
                                  function=brightthresh), 
                         name='brightthresh_filt')    
    
smooth_filt = Node(SUSAN(fwhm=fwhm), name='smooth_filt')

motion_summary = Node(Function(input_names=['motion_df_file','motion_file','vols_to_censor'], 
                               output_names=[], 
                               function=summarize_motion), 
                      name='motion_summary')
motion_summary.inputs.motion_df_file = motion_df_file

In [ ]:
# workflowname.connect([(node1,node2,[('node1output','node2input')]),
#                       (node2,node3,[('node2output','node3input')])
#                     ])

rs_procwf = Workflow(name='rs_procwf')
rs_procwf.connect([(infosource,selectfiles,[('subject_id','subject_id')]),
                   (selectfiles,compcor,[('func','realigned_file')]),
                   (selectfiles,merge_confs,[('csf','mask1')]),
                   (selectfiles,merge_confs,[('wm','mask2')]),
                   (merge_confs,compcor,[('vols','mask_files')]),
                   (compcor,noise_mat,[('components_file','comp_noise')]),
                   (selectfiles,noise_mat,[('vols_to_censor','vols_to_censor'),
                                           ('motion_params','motion_params')]),
                   (noise_mat,denoise,[('noise_filepath','design')]),
                   (selectfiles,denoise,[('func','in_file')]),
                   (denoise,bandpass,[('out_data','in_file')]),
                   (bandpass,afni_convert,[('out_file','in_file')]),
                   (afni_convert,brightthresh_filt,[('out_file','func')]),
                   (brightthresh_filt,smooth_filt,[('bright_thresh','brightness_threshold')]),
                   (afni_convert,smooth_filt,[('out_file','in_file')]),  
                   (selectfiles, motion_summary, [('motion_params','motion_file'),
                                                  ('vols_to_censor','vols_to_censor')]),
                   
                   (register_template, datasink,[('out_file','preproc_struct')]),
                   (smooth_filt,datasink,[('smoothed_file','preproc_func')])
                   ])

rs_procwf.base_dir = workflow_dir
rs_procwf.write_graph(graph2use='flat')
rs_procwf.run('MultiProc', plugin_args={'n_procs': proc_cores})

